## 환경설정

In [61]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("obsidian-rag", set_enable=True)

LangSmith 추적을 시작합니다.
[프로젝트명]
obsidian-rag


## 문서 로더

In [3]:
# loader docs
from document_loaders.obsidian import MyObsidianLoader

docs_path = "./obsidian-help/ko"
loader = MyObsidianLoader(docs_path, encoding="utf-8", collect_metadata=True)
documents = loader.load()

In [8]:
print(len(documents))

129


## 임베딩 모델

In [107]:
# embedding
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "BAAI/bge-m3"
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': False}
underlying_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3


## 임베딩 캐시

https://python.langchain.com/docs/how_to/caching_embeddings/

In [109]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, 
    namespace=underlying_embeddings.model_name,
)

## 문서 분할

시멘틱 청크

In [6]:
# text_splitter
from langchain_experimental.text_splitter import SemanticChunker

text_splitter = SemanticChunker(cached_embedder)

splitted_docs = text_splitter.split_documents(documents)
print(splitted_docs[0])
print(len(splitted_docs))

page_content='# 옵시디언 도움말

공식 Obsidian 도움말 사이트에 오신 것을 환영합니다. 여기에서 [Obsidian](https://obsidian.md/) 사용 팁 및 가이드를 찾을 수 있습니다. API 문서는 [Obsidian 개발자 문서](https://docs.obsidian.md/)에서 확인하실 수 있습니다. 다음 언어로도 이 사이트를 둘러보실 수 있습니다:

<select class="dropdown select-location">
<option value="">English</option>
<option value="https://publish.obsidian.md/help-ar">العربية</option>
<option value="https://publish.obsidian.md/help-da">Dansk</option>
<option value="https://publish.obsidian.md/help-es">Español</option>
<option value="https://publish.obsidian.md/help-it">Italiano</option>
<option value="https://publish.obsidian.md/help-ja">日本語</option>
<option value="https://publish.obsidian.md/help-km">Phéasa Khmêr</option>
<option value="https://publish.obsidian.md/help-ko">한국어</option>
<option value="https://publish.obsidian.md/help-pt-br">Português</option>
<option value="https://publish.obsidian.md/help-ru">Русский</option>
<option value="https://publish.obsidian.md/help-vi">Tiếng Việt</option>
<option valu

## 벡터스토어

임베딩 인덱싱

In [9]:
# vector_store
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = cached_embedder

index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [10]:
# indexing embedding
vector_store.add_documents(splitted_docs)

['f2ddda56-74ee-420b-8f7a-4f9bef89ca99',
 'b1f5f5ee-9b22-4027-aa82-eb64484af204',
 '1b644b10-6bfa-48ce-b213-3cf63b02a84e',
 '41bf2a5e-de81-43a8-8365-d11b254450a7',
 'b2d33808-a075-4728-a2fd-9f6baf38e2e4',
 '55d0def8-594c-40bc-a0b5-8a28e47c9a30',
 '886c3120-a4fb-4bd3-a37e-2e485d954efa',
 '6b97a7e8-1873-4fcf-816a-140c1d9fed2c',
 'fbecd1e9-9a82-44d5-b157-66f725fd38c3',
 '3cf5eec5-3b2d-4c7d-b8a8-1ad4cb24ac9d',
 '4f6a49a9-e5a4-4363-b98e-171cc5a8b7b7',
 '9c89dab2-2be0-4025-a9cf-4a490e4d577a',
 'a752c2dc-165e-43c7-85d6-99f7f2f548ac',
 '21ef59cb-4dbd-4e0f-bfcd-7362971588dd',
 '8fb73dea-347e-4d98-85ca-f28b7ba99c84',
 '728b31ff-e435-4105-9900-cfbb157ad5c8',
 '7c7df2e3-d6b3-4288-9c00-6ec16d1f57ff',
 'c3473aec-aa90-4608-b21c-c0d269f69980',
 'fbd39b2b-9c7d-47a2-907c-90c8d2ea728d',
 'bf9dc1a6-1453-47c5-94a7-72d610f8e132',
 'fa39c78d-196a-4894-b4fe-7ede47b60bef',
 '9bc1523e-5a9a-44b1-a030-51b55d5cd0aa',
 'bd731101-0a09-4323-a7b4-453adae1ddbb',
 '233c1dff-8bbd-4085-b551-50ef13e3038a',
 'bc71645e-59ba-

In [11]:
# save vector_store
vector_store.save_local("faiss_index")

In [12]:
vector_store.similarity_search("옵시디언에서 볼드를 입력하는 방법은?", k=1)

[Document(metadata={'source': '설정 폴더.md', 'path': 'obsidian-help/ko/파일 및 폴더/설정 폴더.md', 'created': 1727746136.5369875, 'last_modified': 1727746136.5369875, 'last_accessed': 1727746143.4086661, 'aliases': "['.config', 'Customization/Config folders']"}, page_content='Obsidian의 설정 폴더에는 [[보관소 관리|볼트]]와 관련된 모든 설정 파일이 포함되어 있습니다. 기본적으로, 설정 폴더는 `.obsidian`으로 명명되어 볼트 폴더 안에 위치합니다. 동기화 서비스를 사용하거나, 동일한 볼트에서 다른 프로필을 테스트하고 싶을 때, 설정 폴더를 변경하고 싶을 수 있습니다. ## 설정 폴더 변경하기\n\n설정 폴더를 변경하려면:\n\n1. **설정 → 파일 및 링크**를 엽니다. 2. **설정 폴더 우선순위 전환**에서 마침표(`.`)로 시작하는 프로필 이름을 입력합니다. 예를 들어, `.obsidian-awesome`. 3.')]

In [13]:
vector_store.similarity_search("옵시디언 노트를 싱크하는 방법은?", k=1)

[Document(metadata={'source': '첫 노트 만들기.md', 'path': 'obsidian-help/ko/시작하기/첫 노트 만들기.md', 'created': 1727746136.5357163, 'last_modified': 1727746136.5357163, 'last_accessed': 1727746143.2059207, 'aliases': 'How to/Create notes'}, page_content='\nObsidian에서 노트는 일반 텍스트 파일로 저장되므로 이동성이 뛰어납니다. 노트를 일반 텍스트로 작성하면 Obsidian 자체를 포함하여 어떤 앱이던간에 노트가 오랜 시간 동안 보존될 수 있습니다. 노트를 장기적으로 보관하려는 계획이 있다면 이것은 좋은 소식입니다! ## Create a new note\n\n새로운 노트를 작성하려면 다음 단계를 따르세요:\n\n1. 키보드에서 `Ctrl+N` (또는 macOS에서는 `Cmd+N`)을 누르세요. 2. "Obsidian"을 노트 이름으로 입력하고 `Enter` 키를 누르세요. 3. 다음 텍스트를 노트에 복사하여 붙여넣으세요:\n\n> "Obsidian은 일반 텍스트 마크다운(Markdown) 파일로 이루어진 로컬 폴더 위에 구축된 강력한 지식 베이스입니다."\n\n## Format your note\n\nObsidian은 [마크다운(Markdown)](https://en.wikipedia.org/wiki/Markdown)도 지원합니다. 이는 일반 텍스트 파일에 서식을 추가하는 데 사용되는 마크업 언어입니다. 1. 다음 텍스트를 Obsidian 노트의 맨 위에 복사하여 붙여넣으세요:\n\n   > \\# 두 번째 뇌, 영원한 당신을 위해. 해시태그 (`#`)는 텍스트 행을 제목으로 변환합니다.')]

## BM25

In [28]:
from langchain_community.retrievers import BM25Retriever

bm25_retriever = BM25Retriever.from_documents(splitted_docs, k=1)

In [29]:
bm25_retriever.invoke("옵시디언에서 볼드를 입력하는 방법은?")

[Document(metadata={'source': '검색.md', 'path': 'obsidian-help/ko/플러그인/검색.md', 'created': 1727746136.538845, 'last_modified': 1727746136.538845, 'last_accessed': 1727746143.4981472}, page_content='검색(Search) 플러그인을 사용하면 보관소 내의 파일을 검색할 수 있습니다. 기본적으로 검색은 왼쪽 사이드바(돋보기 아이콘)에서 찾을 수 있습니다. 또한 `Ctrl+Shift+F` (또는 macOS에서 `Cmd+Shift+F`)를 눌러 검색을 열 수도 있습니다. - **선택한 텍스트 검색**: 편집기에서 텍스트를 선택하고 키보드 단축키로 검색을 열면, 검색은 선택한 텍스트에 대한 검색 결과를 표시합니다. - **최근 검색어 검색**: 빈 검색어로 검색을 열면 최근 검색어를 나열합니다. 이 중에서 검색어를 선택하여 검색어를 다시 사용할 수 있습니다. ## Search terms\n\n검색어는 검색 필드에 입력하는 단어 또는 구절을 의미합니다. 효과적인 검색어 작성 방법을 익히면 큰 보관소에서도 빠르게 원하는 것을 찾을 수 있습니다. Obsidian은 노트와 캔버스의 내용만 검색합니다. > [!tip] 경로 및 파일 이름 검색\n> 기본적으로 노트와 캔버스의 경로와 파일 이름만 검색할 수 있습니다. 보관소의 모든 파일의 경로 또는 파일 이름을 검색하려면 `path` 또는 `file` 연산자를 사용하세요. 검색어의 각 단어는 각 파일 내에서 독립적으로 일치합니다. 정확한 구를 검색하려면 따옴표로 묶어줍니다. 예를 들어 `"스타 워즈"`처럼요. 정확한 구절 내에서 따옴표로 묶인 텍스트를 검색하려면 따옴표 앞에 역슬래시(`\\`)를 추가합니다. 예를 들어 `"그들은 \\"안녕\\"이라고 말했다"`.')]

In [32]:
bm25_retriever.invoke("옵시디언 노트를 싱크하는 방법은?")

[Document(metadata={'source': '홈.md', 'path': 'obsidian-help/ko/홈.md', 'created': 1727746136.5413582, 'last_modified': 1727746136.5413582, 'last_accessed': 1727746143.6017084, 'aliases': "['Start here', 'Obsidian/Index']", 'cssclasses': "['list-cards', 'hide-title']"}, page_content='# 옵시디언 도움말\n\n공식 Obsidian 도움말 사이트에 오신 것을 환영합니다. 여기에서 [Obsidian](https://obsidian.md/) 사용 팁 및 가이드를 찾을 수 있습니다. API 문서는 [Obsidian 개발자 문서](https://docs.obsidian.md/)에서 확인하실 수 있습니다. 다음 언어로도 이 사이트를 둘러보실 수 있습니다:\n\n<select class="dropdown select-location">\n<option value="">English</option>\n<option value="https://publish.obsidian.md/help-ar">العربية</option>\n<option value="https://publish.obsidian.md/help-da">Dansk</option>\n<option value="https://publish.obsidian.md/help-es">Español</option>\n<option value="https://publish.obsidian.md/help-it">Italiano</option>\n<option value="https://publish.obsidian.md/help-ja">日本語</option>\n<option value="https://publish.obsidian.md/help-km">Phéasa Khmêr</option>\n<option va

## Kiwi BM25

한국어 형태소 분석기 키위(Kiwi)

In [35]:
from langchain_teddynote.retrievers import KiwiBM25Retriever

kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(documents=splitted_docs, k=1)

In [31]:
kiwi_bm25_retriever.invoke("옵시디언에서 볼드를 입력하는 방법은?")

[Document(metadata={'source': '보관소 관리.md', 'path': 'obsidian-help/ko/파일 및 폴더/보관소 관리.md', 'created': 1727746136.5369236, 'last_modified': 1727746136.5369236, 'last_accessed': 1727746143.3503828, 'aliases': "['User interface/Vault switcher', 'How to/Working with multiple vaults', 'Vault switcher']"}, page_content='**목록에서 제거**를 선택합니다. ## Transfer settings to another vault\n\n다른 보관소에서 동일한 설정을 사용하려면 선호하는 파일 관리자 (또는 터미널)를 사용하여 원본 보관소의 루트에서 `.obsidian` 폴더를 대상 보관소의 루트로 복사하십시오. 변경 사항을 적용하려면 Obsidian을 재시작해야 할 수도 있습니다. > [!note] `.obsidian` 폴더는 어디에서 찾을 수 있나요? > 기본적으로 대부분의 운영 체제는 점 (`.`)으로 시작하는 폴더를 숨깁니다. `.obsidian` 폴더 및 해당 폴더에 액세스하는 방법에 대한 자세한 내용은 [[Obsidian이 데이터를 저장하는 방법#Vault settings]]을 참조하십시오. ')]

In [33]:
kiwi_bm25_retriever.invoke("옵시디언 노트를 싱크하는 방법은?")

[Document(metadata={'source': '여러 장치에서 노트 동기화하기.md', 'path': 'obsidian-help/ko/시작하기/여러 장치에서 노트 동기화하기.md', 'created': 1727746136.535341, 'last_modified': 1727746136.535341, 'last_accessed': 1727746143.8404388}, page_content="### Why can't I sync using X? 다른 서비스를 파일 동기화에 사용하는 경우가 많고 이를 사용하여 노트를 동기화하려는 경우를 이해합니다. Obsidian은 iOS에서 다른 Markdown 편집기와는 다르게 작동합니다. 1Writer 및 iA Writer와 같은 편집기는 한 번에 한 노트만 액세스하므로 내장된 문서 지원을 사용할 수 있습니다. 반면 Obsidian의 많은 기능은 보관소 전체에 액세스해야 합니다. 예를 들어 파일 이름을 변경하면 Obsidian은 해당 파일에 링크된 보관소의 모든 파일을 업데이트해야 합니다. 지원되지 않는 위치 외부의 수천 개의 노트로 구성된 전체 폴더 구조를 읽고 수정하고 모니터링하는 시스템을 구현하는 것은 복잡합니다. 이 제한 사항을 앞으로 해결할 계획입니다. 개발자라면 각 개별 동기화 서비스에 대한 Web API를 사용하는 플러그인을 빌드할 수 있습니다. ### Where are my vaults stored? 보관소를 만들 때 iCloud Drive를 사용하지 않을 경우 Obsidian은 Obsidian 앱의 로컬 파일 시스템에 저장합니다. 다른 앱인 [[#Working Copy]]과 같은 앱은 파일 시스템에서 보관소를 선택하여 액세스할 수 있습니다. **주의:** 로컬 파일 시스템에 저장된 노트는 Obsidian 앱을 삭제할 때 iOS에서 삭제됩니다. 앱을 삭제하기 전에 노트를 백업해야 합니다. ## Sync notes on Android\n\nAndroid 장치에서 노트를 동기화하는 가장 쉬운 방법은 [[Ob

## 앙상블 리트리버(EnsembleRetriever)

In [39]:
from langchain.retrievers import EnsembleRetriever
vectorstore_retriever = vector_store.as_retriever(search_kwargs={"k": 20})

kiwi_bm25_retriever.k = 20
retriever = EnsembleRetriever(
    retrievers=[kiwi_bm25_retriever, vectorstore_retriever],
    weights=[0.6, 0.4],
    search_type="mmr",
)

In [53]:
retriever.invoke("옵시디언 노트를 싱크하는 방법은?")

[Document(metadata={'source': '여러 장치에서 노트 동기화하기.md', 'path': 'obsidian-help/ko/시작하기/여러 장치에서 노트 동기화하기.md', 'created': 1727746136.535341, 'last_modified': 1727746136.535341, 'last_accessed': 1727746143.8404388}, page_content="### Why can't I sync using X? 다른 서비스를 파일 동기화에 사용하는 경우가 많고 이를 사용하여 노트를 동기화하려는 경우를 이해합니다. Obsidian은 iOS에서 다른 Markdown 편집기와는 다르게 작동합니다. 1Writer 및 iA Writer와 같은 편집기는 한 번에 한 노트만 액세스하므로 내장된 문서 지원을 사용할 수 있습니다. 반면 Obsidian의 많은 기능은 보관소 전체에 액세스해야 합니다. 예를 들어 파일 이름을 변경하면 Obsidian은 해당 파일에 링크된 보관소의 모든 파일을 업데이트해야 합니다. 지원되지 않는 위치 외부의 수천 개의 노트로 구성된 전체 폴더 구조를 읽고 수정하고 모니터링하는 시스템을 구현하는 것은 복잡합니다. 이 제한 사항을 앞으로 해결할 계획입니다. 개발자라면 각 개별 동기화 서비스에 대한 Web API를 사용하는 플러그인을 빌드할 수 있습니다. ### Where are my vaults stored? 보관소를 만들 때 iCloud Drive를 사용하지 않을 경우 Obsidian은 Obsidian 앱의 로컬 파일 시스템에 저장합니다. 다른 앱인 [[#Working Copy]]과 같은 앱은 파일 시스템에서 보관소를 선택하여 액세스할 수 있습니다. **주의:** 로컬 파일 시스템에 저장된 노트는 Obsidian 앱을 삭제할 때 iOS에서 삭제됩니다. 앱을 삭제하기 전에 노트를 백업해야 합니다. ## Sync notes on Android\n\nAndroid 장치에서 노트를 동기화하는 가장 쉬운 방법은 [[Ob

## 리랭커(Rerank)

FlashrankRerank

In [58]:
# Rerank
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

compressor = FlashrankRerank(model="ms-marco-MultiBERT-L-12", top_n=3)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)

In [59]:
compression_retriever.invoke("옵시디언 노트를 싱크하는 방법은?")

[Document(metadata={'id': 30, 'relevance_score': 0.9996191, 'source': '다중 커서.md', 'path': 'obsidian-help/ko/편집 및 서식 지정/다중 커서.md', 'created': 1727746136.5377362, 'last_modified': 1727746136.5377362, 'last_accessed': 1727746143.3665502, 'aliases': "['How to/Working with multiple cursors']"}, page_content='\nObsidian을 사용하면 여러 커서를 사용하여 동시에 여러 위치에서 텍스트를 편집할 수 있습니다. 추가 커서를 추가하려면 `Alt` 키 (또는 macOS에서는 `Option` 키)를 누른 상태에서 노트 내의 다른 위치를 클릭하면 됩니다. 모든 추가 커서와 함께 선택을 제거하려면 어떤 키도 누르지 않고 노트 내 어디든 클릭하면 됩니다. 또한 `Escape` 키를 눌러 선택을 제거할 수도 있습니다. ## Rectangular selection\n\n텍스트의 연속적인 줄을 편집하려는 경우 - 예를 들어 단락을 목록 항목으로 바꾸려는 경우 - 드래그하는 동안 `Shift+Alt` (또는 macOS에서는 `Shift+Option`) 키를 누를 수 있습니다. 또는 중간 마우스 버튼을 누른 상태로 드래그할 수도 있습니다.'),
 Document(metadata={'id': 18, 'relevance_score': 0.9995727, 'source': '기본 서식 구문.md', 'path': 'obsidian-help/ko/편집 및 서식 지정/기본 서식 구문.md', 'created': 1727746136.5375443, 'last_modified': 1727746136.5375443, 'last_accessed': 1727746143.3936386, 'aliases': "['How to/Format your notes', 'Mark

In [62]:
# Jina
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import JinaRerank

compressor = JinaRerank(model="jina-reranker-v2-base-multilingual", top_n=3)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)

compression_retriever.invoke("옵시디언 노트를 싱크하는 방법은?")

[Document(metadata={'source': '오디오 레코더.md', 'path': 'obsidian-help/ko/플러그인/오디오 레코더.md', 'created': 1727746136.5398052, 'last_modified': 1727746136.5398052, 'last_accessed': 1727746143.5807981, 'relevance_score': 0.7606506943702698}, page_content='## Record audio in a note\n\n1. Obsidian에서 새 노트를 만들거나 기존 노트를 엽니다. 2. 리본(데스크톱 버전의 왼쪽 사이드바)에서 **녹음 시작/중지** (마이크 아이콘)를 클릭하여 녹음을 시작합니다. 녹음은 아이콘이 색깔이 바뀌는 즉시 시작됩니다. 3. 녹음을 완료하고 노트에 추가하려면 아이콘을 다시 클릭하십시오. Obsidian은 녹음한 오디오 파일을 보관소에 저장하며 활성 노트의 끝에 그것을 [[파일 삽입|임베드]]합니다. > [!note]\n> 오디오 녹음은 노트에서 제거한 후에도 보관소에 남습니다. 컴퓨터에서 녹음을 완전히 제거하려면 Obsidian [[파일 탐색기|파일 탐색기]]를 사용하여 해당 파일을 삭제할 수 있습니다. '),
 Document(metadata={'source': 'Obsidian Sync 소개.md', 'path': 'obsidian-help/ko/Obsidian Sync/Obsidian Sync 소개.md', 'created': 1727746136.5296974, 'last_modified': 1727746136.5296974, 'last_accessed': 1727746143.3155391, 'aliases': "['Obsidian Sync']", 'relevance_score': 0.735641598701477}, page_content='\n[Obsidian Sync](https://obsidian.md/sync)는 Obsidian의 서버에 노트를 저장하

In [63]:
# Cohere reranker
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-multilingual-v3.0")

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)

compression_retriever.invoke("옵시디언 노트를 싱크하는 방법은?")

INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"


[Document(metadata={'source': 'Obsidian Sync 소개.md', 'path': 'obsidian-help/ko/Obsidian Sync/Obsidian Sync 소개.md', 'created': 1727746136.5296974, 'last_modified': 1727746136.5296974, 'last_accessed': 1727746143.3155391, 'aliases': "['Obsidian Sync']", 'relevance_score': 0.9996527}, page_content='\n[Obsidian Sync](https://obsidian.md/sync)는 Obsidian의 서버에 노트를 저장하고 여러 기기 간에 동기화할 수 있게 해주는 클라우드 기반 서비스입니다. > [!important]\n> Obsidian Sync를 Dropbox, Google Drive 또는 OneDrive와 같은 다른 클라우드 스토리지 공급자와 함께 사용하는 경우, [[Obsidian Sync and third-party services]]를 참조하여 동기화 충돌을 피하세요. Obsidian Sync에 대해 자세히 알아보세요:\n\n- Obsidian Sync 사용자가 처음이라면 [[Obsidian Sync 설정]] 방법을 배워보세요. - 무엇을 동기화할지 구성하려면 [[동기화할 파일 및 설정 선택]]를 참조하세요. - 다른 기기에서 Obsidian Sync를 설정하는 방법을 알고 싶다면 [[다른 기기에서 Obsidian Sync 동기화 설정]]을 참조하세요. - 노트의 이전 버전을 보거나 복원하는 방법을 알고 싶다면 [[버전 내역|버전 내역]]를 참조하세요. - 노트를 안전하게 보호하는 방법을 알고 싶다면 [[Obsidian Sync/보안 및 개인정보 보호|보안과 프라이버시]]를 참조하세요. - 본인의 보관소를 백업하는 데 다른 서드파티 서비스를 사용하려면 [[Obsidian Sync and third-party services]]를

In [64]:
# BAAI/bge-reranker-v2-m3
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
compressor = CrossEncoderReranker(model=model, top_n=3)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)

compression_retriever.invoke("옵시디언 노트를 싱크하는 방법은?")

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: mps
Batches: 100%|██████████| 2/2 [00:54<00:00, 27.24s/it]


[Document(metadata={'source': 'Obsidian Sync 소개.md', 'path': 'obsidian-help/ko/Obsidian Sync/Obsidian Sync 소개.md', 'created': 1727746136.5296974, 'last_modified': 1727746136.5296974, 'last_accessed': 1727746143.3155391, 'aliases': "['Obsidian Sync']"}, page_content='\n[Obsidian Sync](https://obsidian.md/sync)는 Obsidian의 서버에 노트를 저장하고 여러 기기 간에 동기화할 수 있게 해주는 클라우드 기반 서비스입니다. > [!important]\n> Obsidian Sync를 Dropbox, Google Drive 또는 OneDrive와 같은 다른 클라우드 스토리지 공급자와 함께 사용하는 경우, [[Obsidian Sync and third-party services]]를 참조하여 동기화 충돌을 피하세요. Obsidian Sync에 대해 자세히 알아보세요:\n\n- Obsidian Sync 사용자가 처음이라면 [[Obsidian Sync 설정]] 방법을 배워보세요. - 무엇을 동기화할지 구성하려면 [[동기화할 파일 및 설정 선택]]를 참조하세요. - 다른 기기에서 Obsidian Sync를 설정하는 방법을 알고 싶다면 [[다른 기기에서 Obsidian Sync 동기화 설정]]을 참조하세요. - 노트의 이전 버전을 보거나 복원하는 방법을 알고 싶다면 [[버전 내역|버전 내역]]를 참조하세요. - 노트를 안전하게 보호하는 방법을 알고 싶다면 [[Obsidian Sync/보안 및 개인정보 보호|보안과 프라이버시]]를 참조하세요. - 본인의 보관소를 백업하는 데 다른 서드파티 서비스를 사용하려면 [[Obsidian Sync and third-party services]]를 참조하세요. - Obsidian Sync에 문제가 있

In [65]:
# upskyy/ko-reranker-8k
# - ko-reranker-8k는 BAAI/bge-reranker-v2-m3 모델에 한국어 데이터를 finetuning 한 model 
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder

model = HuggingFaceCrossEncoder(model_name="upskyy/ko-reranker-8k")
compressor = CrossEncoderReranker(model=model, top_n=3)

compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=retriever,
)

compression_retriever.invoke("옵시디언 노트를 싱크하는 방법은?")

INFO:sentence_transformers.cross_encoder.CrossEncoder:Use pytorch device: mps
Batches: 100%|██████████| 2/2 [00:41<00:00, 20.71s/it]


[Document(metadata={'source': 'Obsidian Sync 소개.md', 'path': 'obsidian-help/ko/Obsidian Sync/Obsidian Sync 소개.md', 'created': 1727746136.5296974, 'last_modified': 1727746136.5296974, 'last_accessed': 1727746143.3155391, 'aliases': "['Obsidian Sync']"}, page_content='\n[Obsidian Sync](https://obsidian.md/sync)는 Obsidian의 서버에 노트를 저장하고 여러 기기 간에 동기화할 수 있게 해주는 클라우드 기반 서비스입니다. > [!important]\n> Obsidian Sync를 Dropbox, Google Drive 또는 OneDrive와 같은 다른 클라우드 스토리지 공급자와 함께 사용하는 경우, [[Obsidian Sync and third-party services]]를 참조하여 동기화 충돌을 피하세요. Obsidian Sync에 대해 자세히 알아보세요:\n\n- Obsidian Sync 사용자가 처음이라면 [[Obsidian Sync 설정]] 방법을 배워보세요. - 무엇을 동기화할지 구성하려면 [[동기화할 파일 및 설정 선택]]를 참조하세요. - 다른 기기에서 Obsidian Sync를 설정하는 방법을 알고 싶다면 [[다른 기기에서 Obsidian Sync 동기화 설정]]을 참조하세요. - 노트의 이전 버전을 보거나 복원하는 방법을 알고 싶다면 [[버전 내역|버전 내역]]를 참조하세요. - 노트를 안전하게 보호하는 방법을 알고 싶다면 [[Obsidian Sync/보안 및 개인정보 보호|보안과 프라이버시]]를 참조하세요. - 본인의 보관소를 백업하는 데 다른 서드파티 서비스를 사용하려면 [[Obsidian Sync and third-party services]]를 참조하세요. - Obsidian Sync에 문제가 있

In [66]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="anpigon/qwen2.5-7b-instruct-kowiki:q8_0", temperature=0)

llm.invoke("안녕!")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


AIMessage(content='안녕하세요! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={'model': 'anpigon/qwen2.5-7b-instruct-kowiki:q8_0', 'created_at': '2024-10-04T02:33:51.383095Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 23088056666, 'load_duration': 19308699041, 'prompt_eval_count': 35, 'prompt_eval_duration': 1872357000, 'eval_count': 12, 'eval_duration': 1407428000}, id='run-84a7bdb8-5f30-4797-820c-a096b5fea911-0', usage_metadata={'input_tokens': 35, 'output_tokens': 12, 'total_tokens': 47})

In [67]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [75]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.retrievers import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

prompt = PromptTemplate.from_template("""
You are a helpful, respectful and honest assistant for question-answering tasks. 
Always answer as helpfully as possible, while being safe.  
Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal information. 
Please ensure that your responses are socially unbiased and positive in nature.
Use the following pieces of retrieved CONTEXT to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer me in Korean. 

<CONTEXT>
{context}      
</CONTEXT>       

Question: {question}
Answer:                                                                                                                                  
""")

compressor = CohereRerank(model="rerank-multilingual-v3.0", top_n=20)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [76]:
print(rag_chain.invoke("옵시디언 노트를 싱크하는 방법은?"))

INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Obsidian에서 노트를 Sync하는 과정은 다음과 같습니다:

1. **Obsidian Sync 계정 생성**: Obsidian 웹사이트(https://obsidian.md/sync)에서 무료 계정을 만듭니다.
2. **Obsidian 앱 설정**: Obsidian 앱의 **설정** 페이지로 이동하고 **동기화** 섹션을 클릭합니다.
3. **Sync 활성화**: **Obsidian Sync 활성화** 스위치를 켭니다.
4. **계정 연결**: 이전 단계에서 생성한 Obsidian Sync 계정 정보(이메일 주소와 암호)를 입력합니다.
5. **클라우드 스토리지 선택 (선택 사항)**: 원격 백업을 위해 Google Drive, Dropbox, OneDrive 등의 클라우드 스토리지를 연결할 수 있습니다.
6. **Sync 폴더 설정**: 로컬 컴퓨터의 Obsidian 노트 데이터 파일이 저장된 디렉토리를 Sync 폴더로 지정합니다.
7. **변경 사항 저장**: 노트에 변경 사항을 만들면 Obsidian은 자동으로 로컬 컴퓨터와 선택한 클라우드 스토리지(있는 경우)에 동기화합니다.

Sync가 활성화되면:

- 여러 기기에서 동일한 노트 데이터베이스에 액세스할 수 있습니다.
- 한 기기에서 노트를 편집하면 다른 모든 Sync된 기기에서도 실시간으로 변경 사항이 반영됩니다.
- 로컬 컴퓨터의 노트 백업과 클라우드 스토리지의 백업을 둘 다 유지할 수 있습니다.

Sync 설정은 언제든지 **설정 > 동기화**에서 확인하고 수정할 수 있습니다. 

참고로, 옵시디언 무료 버전에서는 한 달에 최대 1GB까지 Sync할 수 있고, 유료 구독(월 $4.99)으로 저장 용량 제한을 해제할 수 있습니다.


In [77]:
llm = ChatOllama(model="antegral/llama-varco", temperature=0)
rag_chain = (
    {"context": compression_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
print(rag_chain.invoke("옵시디언 노트를 싱크하는 방법은?"))

INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Obsidian에서 노트를 Sync하는 과정은 다음과 같습니다:

1. **Obsidian Sync 계정 생성**: Obsidian 웹사이트(https://obsidian.md/sync)에서 무료 계정을 만듭니다.
2. **Obsidian 앱에서 설정 메뉴 열기**: Windows, macOS 또는 Linux에서 각각의 경우 Ctrl+/, Cmd+/ 또는 단축키를 사용하여 설정 메뉴를 엽니다.
3. **Sync 옵션 선택**: "Sync" 섹션으로 이동한 다음 "Sync enabled" 체크박스를 활성화합니다.
4. **Sync 계정 연결**: "Sync account" 아래에서 "Connect to Sync account" 버튼을 클릭하고 이전에 만든 Obsidian Sync 계정의 이메일 주소와 암호를 입력합니다.
5. **원격 보관소 선택 (선택 사항)**: 원격 보관소를 로컬 저장소와 함께 사용할 경우, "Remote vaults" 섹션에서 추가 보관소를 연결할 수 있습니다.
6. **Sync 시작**: 설정 메뉴를 닫으면 Obsidian은 자동으로 모든 개방된 노트를 초기 Sync에 전송합니다.

이후 변경 사항은 다음과 같이 실시간으로 Sync됩니다:

- 노트 편집
- 새로운 노트 또는 페이지 생성
- 파일 이름 변경
- 폴더 구조 수정

Obsidian Sync는 인터넷 연결이 끊기면 로컬 데이터를 캐시하고 다시 온라인 상태가 되면 자동으로 Sync합니다.

주의할 점:
- 옵시디언 프리미엄 구독 없이는 최대 1GB의 저장 공간만 사용 가능합니다.
- 개인 정보 보호를 위해 Obsidian Sync는 암호화된 통신을 사용합니다.
- 원격 보관소에 연결하려면 인터넷 접속이 필요합니다. 
- 로컬 데이터와 원격 보관소 간의 양방향 동기화를 위해 주기적으로 전체 시스템 체크가 좋습니다. 

Sync 문제 해결에 대해서는 [Obsidian 공식 문서](https://docs.obsidian.md/Help#sync-troubleshooting)에서 자세한

In [78]:
print(rag_chain.invoke("옵시디언 노트를 무료료 싱크하는 방법은?"))

INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


Obsidian의 무료 버전인 Obsidian MD에서 노트를 싱크하려면 다음과 같은 옵션이 있습니다:

1. **GitHub repositories 사용하기:** 
   - GitHub 계정에 무료로 존재할 수 있습니다.
   - Obsidian 설정에서 'Sync' > 'Remote Storage' > 'GitHub'을 선택합니다.
   - 저장소 이름과 리포지토리 권한 설정을 합니다.

2. **GitLab repositories 사용하기:**
   - GitLab Community Edition은 무료로 이용 가능합니다.
   - 위의 GitHub 방법과 동일하게 Obsidian 설정에서 GitLab를 선택합니다.

3. **Nextcloud 또는 Self-Hosted WebDAV 서버 사용하기:**
   - Nextcloud와 같은 클라우드 서비스나 자체 호스팅한 WebDAV 서버가 필요합니다.
   - WebDAV 연결 설정을 위해 Obsidian의 'Sync' > 'Remote Storage' > 'WebDAV'로 이동합니다.

4. **OneDrive (Free) 사용하기:**
   - OneDrive 기본 저장 공간(5GB)은 무료입니다.
   - Obsidian 설정에서 'Sync' > 'Remote Storage' > 'OneDrive'를 선택하고 계정에 로그인합니다.

이 중 어느 방법을 선택하든, 초기 설정에는 약간의 시간이 걸릴 수 있습니다. 하지만 이후에는 Obsidian이 자동으로 변경 사항을 싱크해줍니다.

참고로, Obsidian MD의 무료 버전에서는 다음과 같은 제한 사항이 있습니다:

- 최대 5개 파일 동시 싱크
- 하루 최대 1GB 데이터 전송량
- 특정 기능(예: 태그 필터링, 그래프 뷰) 제한

Obsidian Pro로 업그레이드하면 이러한 제한을 해제할 수 있습니다.


## 할루시네이션 체크

- 업스테이지 API 키 발급: https://console.upstage.ai/api-keys
- 소스코드: [UpstageGroundednessCheck](https://api.python.langchain.com/en/latest/_modules/langchain_upstage/tools/groundedness_check.html#UpstageGroundednessCheck)

In [80]:
from langchain_upstage import UpstageGroundednessCheck

upstage_groundedness_check = UpstageGroundednessCheck()

In [81]:
request_input = {
    "context": format_docs(
        compression_retriever.invoke("옵시디언 노트를 무료료 싱크하는 방법은?")
    ),
    "answer": """Obsidian의 무료 버전인 Obsidian MD에서 노트를 싱크하려면 다음과 같은 옵션이 있습니다:

1. **GitHub repositories 사용하기:** 
   - GitHub 계정에 무료로 존재할 수 있습니다.
   - Obsidian 설정에서 'Sync' > 'Remote Storage' > 'GitHub'을 선택합니다.
   - 저장소 이름과 리포지토리 권한 설정을 합니다.

2. **GitLab repositories 사용하기:**
   - GitLab Community Edition은 무료로 이용 가능합니다.
   - 위의 GitHub 방법과 동일하게 Obsidian 설정에서 GitLab를 선택합니다.

3. **Nextcloud 또는 Self-Hosted WebDAV 서버 사용하기:**
   - Nextcloud와 같은 클라우드 서비스나 자체 호스팅한 WebDAV 서버가 필요합니다.
   - WebDAV 연결 설정을 위해 Obsidian의 'Sync' > 'Remote Storage' > 'WebDAV'로 이동합니다.

4. **OneDrive (Free) 사용하기:**
   - OneDrive 기본 저장 공간(5GB)은 무료입니다.
   - Obsidian 설정에서 'Sync' > 'Remote Storage' > 'OneDrive'를 선택하고 계정에 로그인합니다.

이 중 어느 방법을 선택하든, 초기 설정에는 약간의 시간이 걸릴 수 있습니다. 하지만 이후에는 Obsidian이 자동으로 변경 사항을 싱크해줍니다.

참고로, Obsidian MD의 무료 버전에서는 다음과 같은 제한 사항이 있습니다:

- 최대 5개 파일 동시 싱크
- 하루 최대 1GB 데이터 전송량
- 특정 기능(예: 태그 필터링, 그래프 뷰) 제한

Obsidian Pro로 업그레이드하면 이러한 제한을 해제할 수 있습니다.""",
}

response = upstage_groundedness_check.invoke(request_input)
print(response) # grounded 또는 notGrounded, notSure

INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.upstage.ai/v1/solar/chat/completions "HTTP/1.1 200 OK"


notSure


## 프롬프트 개선하기

사용자 질문의 의도를 분석하여 프롬프트를 더 풍부하게 만들어보자.

In [87]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.output_parsers import StrOutputParser


def prompt_enhancer(query: str, llm: BaseChatModel):
    prompt = PromptTemplate.from_template("""You are a highly specialised psychologist and linguist, and your task is to predict my intentions and refine your prompts for the most effective responses.
Edit the prompt to be more detailed and form complete sentences.
Write as concise and short as possible in Korean.
Whenever you can create a role, situation, or task within a prompt, do so,
The prompts you type should be limited to four sentences or less. Only type your own answers.

Original prompt: {query}                                                                 
Improved prompt:
""")
    chain = (
        {
            "query": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
    )
    return chain.invoke(query)

In [88]:
prompt_enhancer("옵시디언 노트를 무료료 싱크하는 방법은?", ChatOllama(model="antegral/llama-varco"))

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


"옵시디언(독을 보는 자의 행동에 관한 메타필로소피) 저자 어린 티베도의 에세이를 PDF 형태로 무료로 얻고 싶다면, 아래 몇 가지 방법을 시도해 볼 수 있습니다.\n\n1. 인터넷 도서관 이용: ProQuest, JSTOR, Google Scholar 등의 학술 데이터베이스에는 옵시디언의 전쇄본이 포함되어 있을 수 있습니다. 일부는 무료 접근을 제공하지만 등록이나 대학/기관 소속증명이 필요한 경우도 있습니다.\n\n2. 저작권 종료 공유: 대부분의 국가에서는 특정 연령(예: 50~70년) 후에는 저작권이 자동으로 종료됩니다. 따라서 옵시디언 원작이 저작권이 종료된 경우 일부 디지털 도서관이나 웹사이트에서 무료로 PDF 형태로 제공받을 수 있을지도 모릅니다.\n\n3. 오픈 액세스 저널: PLOS One, BioMed Central 등 일부 학술 저널은 옵시디언에 관한 연구를 포함한 모든 콘텐츠를 '오픈 액세스'로 제공합니다. 이들 저널의 검색 기능을 활용해 원고를 찾아볼 수 있습니다.\n\n4. 아카이브.org 활용: Internet Archive(archive.org)는 저작권 침해가 없는 책, 논문 등 다양한 서적을 무료 디지털 형태로 보관하고 있습니다. 옵시디언과 관련된 자료가 있으면 PDF로 다운로드 가능할 것입니다.\n\n5. 독립 출판사나 연구기관 웹사이트 확인: 어린 티베도의 저작권이 아직 유효한 상황이라면, 출판사나 연구 기관의 공식 웹사이트에서 무료 샘플 챕터 또는 에세이를 제공하는 경우가 있을 수 있습니다.\n\n주의 사항:\n- 공공 도서관 이용 권한도 확인해 보시기 바랍니다. 일부 도서관에서는 회원 자격으로 온라인 접근이 가능할 것입니다.\n- 불법 복제 행위는 저작권 침해로 여겨질 수 있으므로 주의해야 합니다.\n- 무료 PDF 형태가 제공되는지 여부는 출판사의 정책에 달려 있기 때문에 모든 출판사에서 동일하게 이용 가능한 것은 아닙니다. \n- 필요한 경우 저자 혹은 출판사에게 직접 문의해 볼 수도 있습니다. 일부 사례에서는 한정된 

In [89]:
prompt_enhancer("옵시디언 노트를 무료료 싱크하는 방법은?", ChatOllama(model="anpigon/qwen2.5-7b-instruct-kowiki:q8_0"))

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


'옵시디언 노트를 무료로 동기화할 수 있는 방법은 무엇인가요?'

In [90]:
prompt_enhancer("옵시디언 노트를 무료료 싱크하는 방법은?", ChatOllama(model="eeve-korean-instruct-10.8b:latest"))

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


'옵시디언 노트(Obsidian Notes)를 무료로 동기화하는 방법을 알려줄 수 있나요?'

In [93]:
query = "옵시디언에서 노트를 태그로 검색하려면?"
print('qwen2.5-7b-instruct-kowiki:', prompt_enhancer(query, ChatOllama(model="anpigon/qwen2.5-7b-instruct-kowiki:q8_0")))
print("*" * 20)
print('eeve-korean-instruct-10.8b:', prompt_enhancer(query, ChatOllama(model="eeve-korean-instruct-10.8b:latest")))

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


qwen2.5-7b-instruct-kowiki: 옵시디언에서 특정 태그로 노트를 검색하려면 어떻게 해야 하나요?
********************


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


eeve-korean-instruct-10.8b: 옵션스티안에서 노트를 태그로 검색하려면, 다음 단계를 따르세요:

1. 옵션스티안에서 노트 목록을 열거나 새로 생성하세요.
2. 원하는 태그를 입력하여 검색창에 입력하세요.
3. '태그 추가' 또는 '태그 편집' 버튼을 클릭하면 노트에 해당 태그가 자동으로 적용됩니다.
4. 태그 이름을 변경하거나 다른 노트를 추가하려면, 각 노트의 오른쪽 상단에 있는 옵션을 클릭한 후 '태그 수정'을 선택하세요.

특정 키워드나 구문을 검색하고 싶다면, '전체 텍스트' 필드를 사용하여 더 정확한 결과를 얻으실 수 있습니다.


## LLM Cache

https://python.langchain.com/docs/how_to/chat_model_caching/

In [99]:
from langchain_core.globals import set_llm_cache
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

In [100]:
%time
llm.invoke("Tell me a joke")

CPU times: user 2 μs, sys: 1e+03 ns, total: 3 μs
Wall time: 4.77 μs


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


AIMessage(content='What do you call a fake noodle?\n\nAn impasta.', additional_kwargs={}, response_metadata={'model': 'antegral/llama-varco', 'created_at': '2024-10-04T05:19:53.113391Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 23327359417, 'load_duration': 22004043875, 'prompt_eval_count': 13, 'prompt_eval_duration': 132623000, 'eval_count': 14, 'eval_duration': 1189076000}, id='run-6539f85d-44ba-4751-bae7-54049207b206-0', usage_metadata={'input_tokens': 13, 'output_tokens': 14, 'total_tokens': 27})

In [101]:
%time
llm.invoke("Tell me a joke")

CPU times: user 3 μs, sys: 1e+03 ns, total: 4 μs
Wall time: 6.91 μs


AIMessage(content='What do you call a fake noodle?\n\nAn impasta.', additional_kwargs={}, response_metadata={'model': 'antegral/llama-varco', 'created_at': '2024-10-04T05:19:53.113391Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 23327359417, 'load_duration': 22004043875, 'prompt_eval_count': 13, 'prompt_eval_duration': 132623000, 'eval_count': 14, 'eval_duration': 1189076000}, id='run-6539f85d-44ba-4751-bae7-54049207b206-0', usage_metadata={'input_tokens': 13, 'output_tokens': 14, 'total_tokens': 27})

### SQLiteCache

In [102]:
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [103]:
%time
llm.invoke("Tell me a joke")

CPU times: user 1e+03 ns, sys: 1 μs, total: 2 μs
Wall time: 5.25 μs


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


AIMessage(content='What do you call a fake noodle?\n\nAn impasta.', additional_kwargs={}, response_metadata={'model': 'antegral/llama-varco', 'created_at': '2024-10-04T05:21:49.414198Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2893230834, 'load_duration': 32641084, 'prompt_eval_count': 13, 'prompt_eval_duration': 1696866000, 'eval_count': 14, 'eval_duration': 1162872000}, id='run-9e707d9f-811b-4b9e-a120-74561c54e220-0', usage_metadata={'input_tokens': 13, 'output_tokens': 14, 'total_tokens': 27})

In [106]:
%time
llm.invoke("Tell me a joke")

CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 μs
Wall time: 3.1 μs


AIMessage(content='What do you call a fake noodle?\n\nAn impasta.', additional_kwargs={}, response_metadata={'model': 'antegral/llama-varco', 'created_at': '2024-10-04T05:21:49.414198Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 2893230834, 'load_duration': 32641084, 'prompt_eval_count': 13, 'prompt_eval_duration': 1696866000, 'eval_count': 14, 'eval_duration': 1162872000}, id='run-9e707d9f-811b-4b9e-a120-74561c54e220-0', usage_metadata={'input_tokens': 13, 'output_tokens': 14, 'total_tokens': 27})